# Lab | Data Structuring and Combining Data

## Challenge 1: Combining & Cleaning Data

In this challenge, we will be working with the customer data from an insurance company, as we did in the two previous labs. The data can be found here:
- https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file1.csv

But this time, we got new data, which can be found in the following 2 CSV files located at the links below.

- https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file2.csv
- https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file3.csv

Note that you'll need to clean and format the new data.

Observation:
- One option is to first combine the three datasets and then apply the cleaning function to the new combined dataset
- Another option would be to read the clean file you saved in the previous lab, and just clean the two new files and concatenate the three clean datasets

In [6]:
import pandas as pd
df=pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file1.csv")
df1=pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file2.csv")
df2=pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file3.csv")

In [17]:
df1=pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file2.csv")

In [7]:
df.columns=df.columns.str.lower()
df.columns=df.columns.str.replace(" ","_")
df.columns=df.columns.str.replace("st","state")
df.columns=df.columns.str.replace("custateomer","customer")
def gender_updating(x):
    if x in ["F", "Femal", "female"]:
        return "F"
    elif x in ["M", "Male"]:
        return "M"
df["gender"] = df["gender"].apply(gender_updating)
df["state"]=df["state"].replace({"Cali":"California","WA":"Washington","AZ":"Arizona"})
df["education"]=df["education"].replace("Bachelor","Bachelors")
df["customer_lifetime_value"]=df["customer_lifetime_value"].str.replace("%","",regex=False)
df["vehicle_class"]=df["vehicle_class"].replace({"Sports Car":"Luxury","Luxury SUV":"Luxury","Luxury Car":"Luxury"})
df['number_of_open_complaints'] = (
    df['number_of_open_complaints']
    .astype(str)                     
    .str.split('/')                  
    .str[1]                          
    .astype(float)                   
)
df['customer_lifetime_value'] = pd.to_numeric(df['customer_lifetime_value'], errors='coerce')
df.fillna(method="ffill",inplace=True)
df.gender.fillna(df["gender"].mode(),inplace=True)
df["customer_lifetime_value"].fillna(df['customer_lifetime_value'].mean(), inplace=True)
df.reset_index(drop=True,inplace=True)

C:\Users\prodd\AppData\Local\Temp\ipykernel_11080\2442106518.py:23: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method="ffill",inplace=True)
C:\Users\prodd\AppData\Local\Temp\ipykernel_11080\2442106518.py:24: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df.gender.fillna(df["gender"].mode(),inplace=True)
C:\Users\prodd\AppData\Local\Temp\ipykernel_11080\2442106518.py:25: FutureWarning: A value is trying to be set on a copy o

In [15]:
df1


,customer,state,gender,education,customer_lifetime_value,income,monthly_premium_auto,number_of_open_complaints,total_claim_amount,policy_type,vehicle_class
0,GS98873,Arizona,F,Bachelors,323912.47%,16061,88,1/0/00,633.600000,Personal Auto,Four-Door Car
1,CW49887,California,F,Master,462680.11%,79487,114,1/0/00,547.200000,Special Auto,SUV
2,MY31220,California,F,College,899704.02%,54230,112,1/0/00,537.600000,Personal Auto,Two-Door Car
3,UH35128,Oregon,F,College,2580706.30%,71210,214,1/1/00,1027.200000,Personal Auto,Luxury Car
4,WH52799,Arizona,F,College,380812.21%,94903,94,1/0/00,451.200000,Corporate Auto,Two-Door Car
...,...,...,...,...,...,...,...,...,...,...,...
991,HV85198,Arizona,M,Master,847141.75%,63513,70,1/0/00,185.667213,Personal Auto,Four-Door Car
992,BS91566,Arizona,F,College,543121.91%,58161,68,1/0/00,140.747286,Corporate Auto,Four-Door Car
993,IL40123,Nevada,F,College,568964.41%,83640,70,1/0/00,471.050488,Corporate Auto,Two-Door Car
994,MY32149,California,F,Master,368672.38%,0,96,1/0/00,28.460568,Personal Auto,Two-Door Car


In [18]:
df1.columns=df1.columns.str.lower()
df1.columns=df1.columns.str.replace(" ","_")
df1.columns=df1.columns.str.replace("st","state")
df1.columns=df1.columns.str.replace("custateomer","customer")
def gender_updating(x):
    if x in ["F", "Femal", "female"]:
        return "F"
    elif x in ["M", "Male"]:
        return "M"
df1["gender"] = df1["gender"].apply(gender_updating)
df1["state"]=df1["state"].replace({"Cali":"California","WA":"Washington","AZ":"Arizona"})
df1["education"]=df1["education"].replace("Bachelor","Bachelors")
df1["customer_lifetime_value"]=df1["customer_lifetime_value"].str.replace("%","",regex=False)
df1["vehicle_class"]=df1["vehicle_class"].replace({"Sports Car":"Luxury","Luxury SUV":"Luxury","Luxury Car":"Luxury"})
df1['number_of_open_complaints'] = (
    df1['number_of_open_complaints']
    .astype(str)                     
    .str.split('/')                  
    .str[1]                          
    .astype(float)                   
)
df1['customer_lifetime_value'] = pd.to_numeric(df1['customer_lifetime_value'], errors='coerce')
df1.fillna(method="ffill",inplace=True)
df1.gender.fillna(df1["gender"].mode(),inplace=True)
df1["customer_lifetime_value"].fillna(df1['customer_lifetime_value'].mean(), inplace=True)
df1.reset_index(drop=True,inplace=True)

C:\Users\prodd\AppData\Local\Temp\ipykernel_11080\3655590124.py:23: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df1.fillna(method="ffill",inplace=True)
C:\Users\prodd\AppData\Local\Temp\ipykernel_11080\3655590124.py:24: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df1.gender.fillna(df1["gender"].mode(),inplace=True)
C:\Users\prodd\AppData\Local\Temp\ipykernel_11080\3655590124.py:25: FutureWarning: A value is trying to be set on a cop

In [26]:
pd.concat([df,df1,df2],ignore_index=True)

,customer,state,gender,education,customer_lifetime_value,income,monthly_premium_auto,number_of_open_complaints,policy_type,vehicle_class,...,State,Customer Lifetime Value,Education,Gender,Income,Monthly Premium Auto,Number of Open Complaints,Policy Type,Total Claim Amount,Vehicle Class
0,RB50392,Washington,F,Master,8.713445e+05,0.0,1000.0,0.0,Personal Auto,Four-Door Car,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,QZ44356,Arizona,F,Bachelors,6.979536e+05,0.0,94.0,0.0,Personal Auto,Four-Door Car,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AI49188,Nevada,F,Bachelors,1.288743e+06,48767.0,108.0,0.0,Personal Auto,Two-Door Car,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,WW63253,California,M,Bachelors,7.645862e+05,0.0,106.0,0.0,Corporate Auto,SUV,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,GA49547,Washington,M,High School or Below,5.363077e+05,36357.0,68.0,0.0,Personal Auto,Four-Door Car,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12069,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,California,23405.987980,Bachelor,M,71941.0,73.0,0.0,Personal Auto,198.234764,Four-Door Car
12070,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,California,3096.511217,College,F,21604.0,79.0,0.0,Corporate Auto,379.200000,Four-Door Car
12071,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,California,8163.890428,Bachelor,M,0.0,85.0,3.0,Corporate Auto,790.784983,Four-Door Car
12072,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,California,7524.442436,College,M,21941.0,96.0,0.0,Personal Auto,691.200000,Four-Door Car


# Challenge 2: Structuring Data

In this challenge, we will continue to work with customer data from an insurance company, but we will use a dataset with more columns, called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis_clean.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by performing data cleaning, formatting, and structuring.

In [34]:
new_df=pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis_clean.csv")
new_df.columns

Index(['unnamed:_0', 'customer', 'state', 'customer_lifetime_value',
       'response', 'coverage', 'education', 'effective_to_date',
       'employmentstatus', 'gender', 'income', 'location_code',
       'marital_status', 'monthly_premium_auto', 'months_since_last_claim',
       'months_since_policy_inception', 'number_of_open_complaints',
       'number_of_policies', 'policy_type', 'policy', 'renew_offer_type',
       'sales_channel', 'total_claim_amount', 'vehicle_class', 'vehicle_size',
       'vehicle_type', 'month'],
      dtype='object')

1. You work at the marketing department and you want to know which sales channel brought the most sales in terms of total revenue. Using pivot, create a summary table showing the total revenue for each sales channel (branch, call center, web, and mail).
Round the total revenue to 2 decimal points.  Analyze the resulting table to draw insights.

2. Create a pivot table that shows the average customer lifetime value per gender and education level. Analyze the resulting table to draw insights.

In [39]:
new_df.pivot_table(index="sales_channel",values=["total_claim_amount"],aggfunc="sum").round(2)

,total_claim_amount
sales_channel,
Agent,1810226.82
Branch,1301204.00
Call Center,926600.82
Web,706600.04


In [38]:
new_df.pivot_table(index="gender",columns="education",values=["customer_lifetime_value"],aggfunc="mean")

customer_lifetime_value                            \
education                Bachelor      College       Doctor   
gender                                                        
F                     7874.269478  7748.823325  7328.508916   
M                     7703.601675  8052.459288  7415.333638   

                                             
education High School or Below       Master  
gender                                       
F                  8675.220201  8157.053154  
M                  8149.687783  8168.832659

## Bonus

You work at the customer service department and you want to know which months had the highest number of complaints by policy type category. Create a summary table showing the number of complaints by policy type and month.
Show it in a long format table.

*In data analysis, a long format table is a way of structuring data in which each observation or measurement is stored in a separate row of the table. The key characteristic of a long format table is that each column represents a single variable, and each row represents a single observation of that variable.*

*More information about long and wide format tables here: https://www.statology.org/long-vs-wide-data/*

In [46]:
new_df.melt(id_vars='policy_type',value_vars="month")

,policy_type,variable,value
0,Corporate Auto,month,2
1,Personal Auto,month,1
2,Personal Auto,month,2
3,Corporate Auto,month,1
4,Personal Auto,month,1
...,...,...,...
10905,Personal Auto,month,1
10906,Personal Auto,month,1
10907,Corporate Auto,month,2
10908,Personal Auto,month,2
